# Cargando datos con API-REST

## ¿Qué es API-REST?

**API** es la **A**pplication **P**rogramming **I**nterface, es decir nuestra forma de interactuar con una aplicación. **REST** quiere decir **RE**presentational **S**tate **T**ransfer. Normalmente cuando hablamos de REST, estamos hablando de una interfaz para obtener datos usando directamente HTTP, es decir es un tipo de web API. 

> **De manera sencilla podemos decir que es una Interfaz con la que se interactúa mediante URLs normalmente para obtener datos.**

La principal diferencia entre esto y una URL común, es que la URL de una página web devolverá algo que tu navegador puede interpretar y mostrar de forma "bonita", mientras que una API web mandará datos o instrucciones útiles para ti o tu ordenador.

In [1]:
from IPython.display import HTML

Veamos la respuesta a una página web clásica

In [2]:
HTML('<iframe src="https://developer.github.com/v3/" width="700" height="400"></iframe>')

Veamos la respuesta de una web api

In [3]:
HTML(url="https://api.github.com/")

En definitiva, cuando utilizamos una WEB API, están involucrados:

* Cliente: realiza la petición a través de la url
* Servidor: da una respuesta a la url recibida
* Protocolo: se establece en la especificación de la API

## ¿Cómo utilizar una API?

El uso de WEB APIs permite: obtener información que sería cosotsa de obtener y procesar de otra manera (incluso en tiempo real).

En algunas ocasiones, la API es pública y cualquiera puede hacer una petición, pero en otras es necesario tener una *api key* que nos identifica. Por lo tanto, el proceso para obtener datos suele ser:

1. Comprobar si existe una API-REST para obtener dichos datos
2. Obtener una api key en caso de que sea necesaria
3. Leer la especificación de la API para saber cómo componer la URL y como interpretar la respuesta
4. Utilizar la API, en nuestro caso desde Python.

### Ejemplo: descargando datos de AEMET 

Esta es la página principal de su API:

[https://opendata.aemet.es/centrodedescargas/inicio](https://opendata.aemet.es/centrodedescargas/inicio)
    
Aquí podemos encontrar: información general, el lugar donde obtener nuestra API key, una interfaz para acceder a los datos para público general 

In [4]:
HTML('<iframe src="https://opendata.aemet.es/centrodedescargas/inicio" width="1000" height="400"></iframe>')

## Módulo requests

Aunque existen otras formas y librería para trabajar con HTTP en Python, el módulo `requests` está específicamente pensado para trabajar con APIs web.

Como siempre hasta ahora, lo primero es importarlo:

In [5]:
import requests

Necesitaremos cargar nuesta API key, lo más cómodo es almacenarla en un fichero y cargarla desde ahí. Creemos una función para leerla:

In [6]:
def load_api_key(file):
    """Returns the contents in the file without the final line break
    """
    with open(file, 'r') as f:
        api_key = f.read().rstrip()
    return api_key

In [7]:
api_key = load_api_key("../../apikey-aemet.txt")

Debemos saber cuál es la url a la que vamos a hacer la petición:

In [8]:
# Predicción costera de Asturias, Cantabria y País Vasco debemos introducir
url = "https://opendata.aemet.es/opendata/api/prediccion/maritima/costera/costa/41"

querystring = {"api_key": api_key}

Por último, lanzamos la petición:

In [9]:
response = requests.get(url, params=querystring, verify=False)

/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Comprobando la respuesta:

In [10]:
response

<Response [200]>

Vemos que hemos obtenido una respuesta con código 200, esto quiere decir, en el código habitual de las API-REST, que todo ha ido bien. De hecho, es conveniente ver que todo ha ido bien antes de hacer nada más:

In [11]:
response.status_code == requests.codes.ok

True

Otra información que podemos obtener es, por ejemplo;

In [12]:
# Tiempo en procesar la petición
print("Elapsed: ", response.elapsed)
# Información del servidor
print("Headers: ", response.headers)

Elapsed:  0:00:00.085245
Headers:  {'Server': 'Apache/2.2.15 (CentOS)', 'Date': 'Thu, 01 Jun 2017 21:06:24 GMT'}


Pero... ¿dónde están nuestros datos?

In [13]:
response.content

b'{\n  "descripcion" : "exito",\n  "estado" : 200,\n  "datos" : "https://opendata.aemet.es/opendata/sh/43cdfb13",\n  "metadatos" : "https://opendata.aemet.es/opendata/sh/8846af34"\n}'

¡Parece que esto es un json!

In [14]:
content = response.json()

content

{'datos': 'https://opendata.aemet.es/opendata/sh/43cdfb13',
 'descripcion': 'exito',
 'estado': 200,
 'metadatos': 'https://opendata.aemet.es/opendata/sh/8846af34'}

Efectivamente, la mayoría de las WEB APIs devuelven json o xml. Pero, otra vez... ¿dónde están nuestros datos?

In [15]:
r_meta = requests.get(content['metadatos'], verify=False)
r_data = requests.get(content['datos'], verify=False)

if r_meta.status_code == requests.codes.ok:
    metadata = r_meta.json()
    
if r_data.status_code == requests.codes.ok:
    data = r_data.json()

/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [16]:
print(metadata[0].keys())
print(data[0].keys())

dict_keys(['unidad_generadora', 'descripcion', 'formato', 'periodicidad', 'campos'])
dict_keys(['aviso', 'situacion', 'origen', 'nombre', 'tendencia', 'prediccion', 'id'])


In [17]:
metadata[0]['campos']

[{'origen': [{'descripcion': 'Agencia Estatal de Meteorología - AEMET. Gobierno de España',
    'id': 'productor',
    'requerido': True,
    'tipo_datos': 'string'},
   {'descripcion': 'URL de la Web de AEMET - http://www.aemet.es',
    'id': 'web',
    'requerido': True,
    'tipo_datos': 'string'},
   {'descripcion': 'es',
    'id': 'language',
    'requerido': True,
    'tipo_datos': 'string'},
   {'descripcion': '© AEMET. Autorizado el uso de la información y su reproducción citando a AEMET como autora de la misma.',
    'id': 'copyright',
    'requerido': True,
    'tipo_datos': 'string'},
   {'descripcion': 'http://www.aemet.es/es/nota_legal/nota_legal',
    'id': 'notaLegal',
    'requerido': True,
    'tipo_datos': 'anyURI'},
   {'descripcion': 'Fecha de elaboración',
    'id': 'elaborado',
    'requerido': True,
    'tipo_datos': 'dataTime'},
   {'descripcion': 'Fecha de inicio de la validez de los avisos',
    'id': 'inicio',
    'requerido': True,
    'tipo_datos': 'dataTim

In [18]:
data[0]['prediccion']

{'fin': '2017-06-02',
 'inicio': '2017-06-01',
 'zona': [{'id': 8033010,
   'nombre': 'Aguas costeras de Asturias',
   'subzona': {'id': 8033010,
    'nombre': 'Aguas costeras de Asturias',
    'texto': 'VARIABLE FUERZA 2, ARRECIANDO A OESTE O NOROESTE 3 O 4 AL ESTE DE PENAS Y 2 O 3 AL OESTE. RIZADA O MAREJADILLA. MAR DE FONDO DEL NOROESTE AUMENTANDO A 1 A 2 METROS. BRUMA, TEMPORALMENTE AREAS DE NIEBLA; AGUACEROS POR LA TARDE. TEMPORALMENTE, REGULAR O MALA.'}},
  {'id': 8063910,
   'nombre': 'Aguas costeras de Cantabria',
   'subzona': {'id': 8063910,
    'nombre': 'Aguas costeras de Cantabria',
    'texto': 'VARIABLE FUERZA 2 ARRECIANDO A OESTE O NOROESTE 2 O 3 DURANTE LA NOCHE, Y A FUERZA 3 O 4 POR LA MANANA. RIZADA O MAREJADILLA, AREAS DE MAREJADA AL OESTE DE MAYOR POR LA MANANA. MAR DE FONDO DEL NOROESTE AUMENTANDO A 1 METRO HACIA LA MADRUGADA Y A 1 A 2 METROS POR LA TARDE. BRUMA, TEMPORALMENTE AREAS DE NIEBLA; AGUACEROS POR LA TARDE. TEMPORALMENTE, REGULAR O LOCALMENTE MALA.'}},
 

En definitiva, podríamos reagrupar todo lo anterior como:

In [19]:
from warnings import warn

def get_prediction_for_cantabria(api_key):
    url = "https://opendata.aemet.es/opendata/api/prediccion/maritima/costera/costa/41"
    querystring = {"api_key": api_key}
    
    response = requests.get(url, params=querystring, verify=False)
    
    prediction = None
     
    if response.status_code == requests.codes.ok:
        r_data = requests.get(content['datos'], verify=False)
        if r_data.status_code == requests.codes.ok:
            data = r_data.json()
            prediction = data[0]['prediccion']
    
    elif response.status_code == requests.codes.TOO_MANY_REQUESTS:
        warn('Too many requests')
        
    elif response.status.code == requests.codes.NOT_FOUND:
        warn('No data for the response')
        
    else:
        warn('Code error {}'.format(response.status_code))
            
    return prediction
        

In [20]:
# Raising too many requests warining
for ii in range(50):
    print(get_prediction_for_cantabria(api_key))

/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'inicio': '2017-06-01', 'fin': '2017-06-02', 'zona': [{'subzona': {'texto': 'VARIABLE FUERZA 2, ARRECIANDO A OESTE O NOROESTE 3 O 4 AL ESTE DE PENAS Y 2 O 3 AL OESTE. RIZADA O MAREJADILLA. MAR DE FONDO DEL NOROESTE AUMENTANDO A 1 A 2 METROS. BRUMA, TEMPORALMENTE AREAS DE NIEBLA; AGUACEROS POR LA TARDE. TEMPORALMENTE, REGULAR O MALA.', 'nombre': 'Aguas costeras de Asturias', 'id': 8033010}, 'nombre': 'Aguas costeras de Asturias', 'id': 8033010}, {'subzona': {'texto': 'VARIABLE FUERZA 2 ARRECIANDO A OESTE O NOROESTE 2 O 3 DURANTE LA NOCHE, Y A FUERZA 3 O 4 POR LA MANANA. RIZADA O MAREJADILLA, AREAS DE MAREJADA AL OESTE DE MAYOR POR LA MANANA. MAR DE FONDO DEL NOROESTE AUMENTANDO A 1 METRO HACIA LA MADRUGADA Y A 1 A 2 METROS POR LA TARDE. BRUMA, TEMPORALMENTE AREAS DE NIEBLA; AGUACEROS POR LA TARDE. TEMPORALMENTE, REGULAR O LOCALMENTE MALA.', 'nombre': 'Aguas costeras de Cantabria', 'id': 8063910}, 'nombre': 'Aguas costeras de Cantabria', 'id': 8063910}, {'subzona': {'texto': 'NORDESTE F

/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3

{'inicio': '2017-06-01', 'fin': '2017-06-02', 'zona': [{'subzona': {'texto': 'VARIABLE FUERZA 2, ARRECIANDO A OESTE O NOROESTE 3 O 4 AL ESTE DE PENAS Y 2 O 3 AL OESTE. RIZADA O MAREJADILLA. MAR DE FONDO DEL NOROESTE AUMENTANDO A 1 A 2 METROS. BRUMA, TEMPORALMENTE AREAS DE NIEBLA; AGUACEROS POR LA TARDE. TEMPORALMENTE, REGULAR O MALA.', 'nombre': 'Aguas costeras de Asturias', 'id': 8033010}, 'nombre': 'Aguas costeras de Asturias', 'id': 8033010}, {'subzona': {'texto': 'VARIABLE FUERZA 2 ARRECIANDO A OESTE O NOROESTE 2 O 3 DURANTE LA NOCHE, Y A FUERZA 3 O 4 POR LA MANANA. RIZADA O MAREJADILLA, AREAS DE MAREJADA AL OESTE DE MAYOR POR LA MANANA. MAR DE FONDO DEL NOROESTE AUMENTANDO A 1 METRO HACIA LA MADRUGADA Y A 1 A 2 METROS POR LA TARDE. BRUMA, TEMPORALMENTE AREAS DE NIEBLA; AGUACEROS POR LA TARDE. TEMPORALMENTE, REGULAR O LOCALMENTE MALA.', 'nombre': 'Aguas costeras de Cantabria', 'id': 8063910}, 'nombre': 'Aguas costeras de Cantabria', 'id': 8063910}, {'subzona': {'texto': 'NORDESTE F

/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3

None
None
None
None


/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3

None
None
None
None
None


/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/asaez/miniconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


---

# Conclusiones

* Si el proveedor de los datos dispone de una API-REST podemos aprovecharla para automatizar la obtención de los mismos
* En la mayoría de los casos necesitaremos una api-key y conocer la especificación de la API
* A través de la URL podemos configurar nuestra petición
* El módulo requests nos permite realizar este proceso en Python de manera secilla

# Enlaces:

* Página oficial de requests: http://docs.python-requests.org/en/master/
* Getting Data Software Carpentry Lesson: http://swcarpentry.github.io/web-data-python/01-getdata/
* Using the Requests Module in Python: https://code.tutsplus.com/tutorials/using-the-requests-module-in-python--cms-28204


---
<br/>
<h4 align="right">¡Síguenos en Twitter!
<br/>
<p align="right"> <a href="https://twitter.com/AeroPython" class="twitter-follow-button" data-show-count="false">@AeroPython</a> <script>!function(d,s,id){var js,fjs=d.getElementsByTagName(s)[0],p=/^http:/.test(d.location)?'http':'https';if(!d.getElementById(id)){js=d.createElement(s);js.id=id;js.src=p+'://platform.twitter.com/widgets.js';fjs.parentNode.insertBefore(js,fjs);}}(document, 'script', 'twitter-wjs');</script> </p>
<p align="right"><a href="https://twitter.com/CAChemEorg" class="twitter-follow-button" data-show-count="false">@CAChemEorg</a> <script>!function(d,s,id){var js,fjs=d.getElementsByTagName(s)[0],p=/^http:/.test(d.location)?'http':'https';if(!d.getElementById(id)){js=d.createElement(s);js.id=id;js.src=p+'://platform.twitter.com/widgets.js';fjs.parentNode.insertBefore(js,fjs);}}(document, 'script', 'twitter-wjs');</script> </p>
<br/></h4>

###### Este notebook ha sido realizado por: [Álex Sáez](https://www.linkedin.com/in/alejandrosaezm/) y [Francisco Navarro](https://www.linkedin.com/in/fjnavarro/)
<br/>
##### <a rel="license" href="http://creativecommons.org/licenses/by/4.0/deed.es"><img alt="Licencia Creative Commons" style="border-width:0" src="http://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Curso de introducción a Python: procesamiento y análisis de datos</span> por <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Juan Luis Cano Rodriguez, Alejandro Sáez Mollejo y Francisco J. Navarro Brull</span> se distribuye bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/deed.es">Licencia Creative Commons Atribución 4.0 Internacional</a>.

###### La mayor parte de material de este curso es un resumen adaptado del magnífico [Curso de AeroPython](https://github.com/AeroPython/Curso_AeroPython)  realizado por: Juan Luis Cano  y Álex Sáez 